In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            (
                "You are a world-class international chef. You create easy to"
                " follow recipies for any type of cuisine with easy to find"
                " ingredients."
            ),
        ),
        ("human", "I want to cook {cuisine} food."),
    ]
)

chef_chain = chef_prompt | chat

veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            (
                "You are a vegetarian chef specialized on making traditional"
                " recipes vegetarian. You find alternative ingredients and"
                " explain their preparation. You don't radically modify the"
                " recipe. If there is no alternative for a food just say you"
                " don't know how to recipe it."
            ),
        ),
        ("human", "{recipe}"),
    ]
)

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({"cuisine": "Korean"})

2024-12-09 17:30:43.501 
  command:

    streamlit run c:\Users\ybr24\FULLSTACK-GPT\env\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


AIMessageChunk(content="For a vegetarian version of Bibimbap, you can easily substitute the beef with tofu or tempeh. Here's how you can prepare the tofu/tempeh as a replacement for the beef in this recipe:\n\nIngredients:\n- 1/2 pound firm tofu or tempeh, thinly sliced\n- 2 tablespoons soy sauce\n- 1 tablespoon sesame oil\n- Pinch of black pepper\n\nInstructions:\n1. Marinate the tofu/tempeh: In a bowl, mix the tofu/tempeh slices with 2 tablespoons of soy sauce, 1 tablespoon of sesame oil, and a pinch of black pepper. Set aside to marinate for at least 15 minutes.\n\n2. Heat a pan over medium heat and add a little vegetable oil. Stir-fry the marinated tofu/tempeh until cooked through. Remove from the pan and set aside.\n\n3. Follow the rest of the instructions in the original recipe to prepare the vegetables, eggs, and assemble the Bibimbap.\n\nBy using tofu or tempeh as a substitute for the beef, you can still enjoy a flavorful and protein-rich Bibimbap while keeping it vegetarian-fr